[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1lSzgEXw9F4X65qSSgOs47ejMGRDkbuZH?usp=sharing)

## Open Source Implementation of NotebookLM

- Open source alternative to Google's NotebookLM
- Uses Deepseek-V3 for language understanding and generation
- Integrates PlayHT for text-to-speech capabilities
- Demonstrates interactive notebook-based AI assistance

Notebook by [Build Fast with AI](https://www.buildfastwithai.com/genai-course)

In [1]:
!pip install -qU fal-client langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.8/454.8 kB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 54.6 MB/s eta 0:00:00


In [3]:
import os
from google.colab import userdata

os.environ["FAL_KEY"] = userdata.get('FAL_KEY')
os.environ["OPENROUTER_API_KEY"] = userdata.get('OPENROUTER_API_KEY')

In [6]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

def generate_podcast_transcript(topic):
    # Modify this template as per your requirement!
    podcast_template = ChatPromptTemplate.from_template("""
      Create an engaging conversation between two speakers discussing the topic: {topic}

      Requirements:
      - Generate exactly 5 back-and-forth exchanges
      - Make it natural and conversational
      - Include specific details about the {topic}
      - Each line should start with either "Speaker 1:" or "Speaker 2:"

      Here's an example of the format (but create NEW content about {topic}, don't copy this example):
      Speaker 1: [First speaker's line]
      Speaker 2: [Second speaker's line]

      The response of the each speaker should be at most 20 words. The conversation has to be insightful, engaging, explanatory, deep diving and educational.

      It should be in the style of a podcast where one speaker slightly is more knowledgeable than the other.

      You are allowed to write only in the below format. Just give the output in the below format in a single string. No additional delimiters.

      The content should be explanatory, deep diving and educational.

      Speaker 1: Hey, did you catch the game last night?
      Speaker 2: Of course! What a match—it had me on the edge of my seat.
      Speaker 1: Same here! That last-minute goal was unreal. Who's your MVP?
      Speaker 2: Gotta be the goalie. Those saves were unbelievable.


      Remember: Create completely new dialogue about {topic}, don't use the above example.
      """)

    # Initialize the ChatOpenAI model
    llm = ChatOpenAI(
        model="deepseek/deepseek-chat",
        openai_api_key=os.getenv("OPENROUTER_API_KEY"),
        openai_api_base="https://openrouter.ai/api/v1"
    )

    # Create the chain
    chain = podcast_template | llm

    response = chain.invoke({"topic": topic})
    return response.content

In [8]:
response = generate_podcast_transcript("Quantum Random Walks")
print(response)

Speaker 1: Have you heard about quantum random walks? They’re like classical random walks but with a quantum twist.  
Speaker 2: Yeah, but how do they differ? Classical walks spread linearly, right?  
Speaker 1: Exactly! Quantum walks spread quadratically faster due to superposition and interference. It’s mind-blowing!  
Speaker 2: So, does that mean they’re more efficient for algorithms?  
Speaker 1: Absolutely! They’re used in quantum search and optimization problems, outperforming classical methods.  
Speaker 2: That’s fascinating! How do they handle decoherence, though?  
Speaker 1: Great question! Decoherence is a challenge, but error correction techniques help maintain quantum coherence.  
Speaker 2: Makes sense. So, are quantum walks practical yet, or still theoretical?  
Speaker 1: Mostly experimental, but progress is rapid. They’re paving the way for quantum computing breakthroughs!  
Speaker 2: Can’t wait to see where this leads. Quantum walks sound like the future!


In [11]:
def generate_podcast(topic):
    print(f"\n🎙️ Generating podcast transcript about: {topic}")
    print("-" * 50)

    # Get transcript first using generate_podcast_transcript
    transcript_result = generate_podcast_transcript(topic)

    print("\n✍️ Generated transcript:")
    print("-" * 50)
    print(transcript_result)

    print("\n🔊 Converting transcript to audio...")
    print("-" * 50)

    # Progress callback for fal-client
    def on_queue_update(update):
        if isinstance(update, fal_client.InProgress):
            for log in update.logs:
                print(f"🎵 {log['message']}")

    # Generate audio using fal-client
    try:
        result = fal_client.subscribe(
            "fal-ai/playht/tts/ldm",
            {
                "input": transcript_result,
                "voices": [
                    {
                        "voice": "Jennifer (English (US)/American)",
                        "turn_prefix": "Speaker 1: "
                    },
                    {
                        "voice": "Dexter (English (US)/American)",
                        "turn_prefix": "Speaker 2: "
                    }
                ]
            },
            with_logs=True,
            on_queue_update=on_queue_update,
        )

        print("\n✅ Audio generation complete!")
        print(f"🔗 Audio URL: {result['audio']['url']}")

        return {
            "conversation": transcript_result,
            "audio_url": result['audio']['url']
        }

    except Exception as e:
        print(f"\n❌ Error generating audio: {str(e)}")
        return {
            "conversation": transcript_result,
            "audio_url": None,
            "error": str(e)
        }

In [12]:
generate_podcast("Quantum Random Walks")


🎙️ Generating podcast transcript about: Quantum Random Walks
--------------------------------------------------

✍️ Generated transcript:
--------------------------------------------------
Speaker 1: Have you heard about quantum random walks? They’re like classical random walks but with quantum weirdness.  
Speaker 2: Yeah, but instead of probabilities, they use amplitudes, right? Superposition makes it way more powerful.  
Speaker 1: Exactly! The walker can explore multiple paths simultaneously, leading to faster spread than classical walks.  
Speaker 2: That’s why they’re used in quantum algorithms, like search problems. The speedup is mind-blowing.  
Speaker 1: True! And the interference effects? They can amplify or cancel paths, making it a game-changer for computation.

🔊 Converting transcript to audio...
--------------------------------------------------

✅ Audio generation complete!
🔗 Audio URL: https://v3.fal.media/files/penguin/TencvNVwGMCgAXACDYIYn_de8c5432-cfb7-4b4b-9f40-e8

{'conversation': 'Speaker 1: Have you heard about quantum random walks? They’re like classical random walks but with quantum weirdness.  \nSpeaker 2: Yeah, but instead of probabilities, they use amplitudes, right? Superposition makes it way more powerful.  \nSpeaker 1: Exactly! The walker can explore multiple paths simultaneously, leading to faster spread than classical walks.  \nSpeaker 2: That’s why they’re used in quantum algorithms, like search problems. The speedup is mind-blowing.  \nSpeaker 1: True! And the interference effects? They can amplify or cancel paths, making it a game-changer for computation.',
 'audio_url': 'https://v3.fal.media/files/penguin/TencvNVwGMCgAXACDYIYn_de8c5432-cfb7-4b4b-9f40-e8c3b819674b.mp3'}